In [3]:
!pip install pygame

In [4]:
import pygame, sys, random
from pygame.locals import *
from fonts import *

BLACK       = (0, 0, 0)
WHITE       = (255, 255, 255)
RED 		= (205, 92, 92)
WIDTH       = 700
HEIGHT      = 500
BALL_RADIUS = 9
PAD_WIDTH   = 10
PAD_HEIGHT  = 70
DIFF 		= 2 
TOPSCORE    = 5

pygame.init()
fpsClock = pygame.time.Clock()
window_obj = pygame.display.set_mode((WIDTH, HEIGHT), 0, 32)
pygame.display.set_caption("Ping Pong")

class Ball():
	def __init__(self, radius, color):
		self.radius = radius
		self.color  = color
	def reset(self, direction):
		self.pos = [int(WIDTH/2),int(HEIGHT/2)]
		if direction == "Right":
			self.vel = [random.randint(2,4),-random.randint(1,3)]
		elif direction == "Left":
			self.vel = [-random.randint(2,4),-random.randint(1,3)]
	def draw(self, window_obj):
		pygame.draw.circle(window_obj, self.color, self.pos, self.radius)
	def update(self):
		if self.pos[1] <= BALL_RADIUS or self.pos[1] >= HEIGHT-BALL_RADIUS:
			self.vel[1] = -self.vel[1]
		self.pos[0] += self.vel[0]
		self.pos[1] += self.vel[1]	

class Paddle():
	def __init__(self, pos, vel, width, height, color, score):
		self.pos    = pos
		self.vel    = vel
		self.width  = width
		self.height = height
		self.color  = color
		self.score  = score
	def draw(self, window_obj):
		pygame.draw.line(window_obj, self.color, (self.pos[0], self.pos[1] - self.height/2), (self.pos[0], self.pos[1] + self.height/2), self.width)
	def update(self):
		if self.pos[1]+self.vel > int(self.height/2) and self.pos[1]+self.vel < int(HEIGHT-self.height/2):
			self.pos[1]=self.pos[1]+self.vel 
	
def check_collision(ball, paddle1, paddle2):
	if ball.pos[0] <= paddle1.width*2 + ball.radius or ball.pos[0] >= WIDTH - paddle1.width*2 - ball.radius:
		if (ball.pos[1] in range(paddle1.pos[1] - int(paddle1.height/2), paddle1.pos[1] + int(paddle1.height/2)) and ball.pos[0] > int(WIDTH/2)) or (ball.pos[1] in range(paddle2.pos[1] - int(paddle2.height/2), paddle2.pos[1] + int(paddle2.height/2)) and ball.pos[0] < int(WIDTH/2)):
			if ball.vel[0]<0:
				ball.vel[0] =- (ball.vel[0]-1)
			else:
				ball.vel[0] =- (ball.vel[0]+1)
		else:
			if ball.pos[0]>int(WIDTH/2):
				paddle1.score +=1
				ball.reset("Left")
			else:
				paddle2.score +=1
				ball.reset("Right")

def win_msg(window_obj, player):
	window_obj.fill(BLACK)
	try:
		font             = pygame.font.Font("fonts/Megadeth.ttf", 70)
	except:
		font             = pygame.font.Font(None, 70)
	msg 			= font.render(player + " Wins", True, WHITE)
	msgRect         = msg.get_rect()
	msgRect.centerx = int(WIDTH/2)
	msgRect.centery = int(HEIGHT/2)
	window_obj.blit(msg, msgRect)
	pygame.display.update()
	while True:	
		for event in pygame.event.get():
			if event.type == QUIT:
				pygame.quit()
				sys.exit()
			elif event.type == KEYDOWN:
				new_game(window_obj)
				return

def draw_scores(window_obj, score1, score2):
	try:
		font          = pygame.font.Font("fonts/impact.ttf", 40)
	except:
		font          = pygame.font.Font(None, 40)
	msg1          = font.render(str(score1), True, WHITE)
	msg2          = font.render(str(score2), True, WHITE)	
	msg1Rect      = msg1.get_rect()
	msg2Rect      = msg2.get_rect()
	msg1Rect.left = int(WIDTH/4)
	msg2Rect.left = int(WIDTH/4 * 3)
	msg1Rect.top  = int(HEIGHT/4)
	msg2Rect.top  = int(HEIGHT/4)
	window_obj.blit(msg1, msg1Rect)
	window_obj.blit(msg2, msg2Rect)

def draw_options(window_obj, highlight):
	try:
		font             = pygame.font.Font("fonts/Megadeth.ttf", 56)
		selected_font	 = pygame.font.Font("fonts/Megadeth.ttf", 70)
		img = pygame.image.load('Untitled.jpg')
		img.convert()
		window_obj.blit(img, [int(WIDTH*3/10), int(HEIGHT*5/10)])
	except:
		font             = pygame.font.Font(None, 56)
		selected_font	 = pygame.font.Font(None, 70)
	opt_list	  	 = [font, selected_font]
	msg1             = opt_list[highlight].render("Ping-Pong", True, WHITE)
	msg2             = opt_list[highlight - 1].render("Press Enter", True, WHITE)
	msg1Rect         = msg1.get_rect()
	msg2Rect         = msg2.get_rect()
	msg1Rect.centerx = int(WIDTH/2)
	msg2Rect.centerx = int(WIDTH/2)
	msg1Rect.centery = int(HEIGHT * 3/10)
	msg2Rect.centery = int(HEIGHT * 4/10)
	window_obj.blit(msg1, msg1Rect)
	window_obj.blit(msg2, msg2Rect)

def run_game(window_obj, ball, paddle1, paddle2):
	ball.update()
	ball.draw(window_obj)
	paddle1.update()
	paddle2.update()
	paddle1.draw(window_obj)
	paddle2.draw(window_obj)
	check_collision(ball, paddle1, paddle2)
	draw_scores(window_obj, paddle1.score, paddle2.score)
	
def new_game(window_obj):
	ball      = Ball(BALL_RADIUS, RED)
	ball.reset("Left")
	paddle1   = Paddle([int(WIDTH-PAD_WIDTH*2),int(HEIGHT/2)], 0, PAD_WIDTH, PAD_HEIGHT, WHITE, 0)
	paddle2   = Paddle([int(PAD_WIDTH*2),int(HEIGHT/2)], 0, PAD_WIDTH, PAD_HEIGHT, WHITE, 0)
	option    = "nil"	
	highlight = 1
	while option == "nil":
		window_obj.fill(BLACK)
		draw_options(window_obj, highlight)
		for event in pygame.event.get():
			if event.type == QUIT:
				pygame.quit()
				sys.exit()
			elif event.type == KEYDOWN:
				if event.key in [K_UP, K_DOWN, K_w, K_s]:
					highlight = highlight ^ 1
				elif event.key == K_RETURN:
					option = bool(highlight ^ 1)
			pygame.display.update()
			fpsClock.tick(60)
	game_loop(window_obj, ball, paddle1, paddle2, option)

def game_loop(window_obj, ball, paddle1, paddle2, one_player):
	while True:
			window_obj.fill(BLACK)
			run_game(window_obj, ball, paddle1, paddle2)
			if paddle1.score >= TOPSCORE:
				win_msg(window_obj, 'Player 1')
				return
			elif paddle2.score >= TOPSCORE:
				win_msg(window_obj, 'Player 2')
				return
			for event in pygame.event.get():
				if event.type == QUIT:
					pygame.quit()
					sys.exit()
				elif event.type == KEYDOWN:	
					if event.key == K_ESCAPE:
						new_game(window_obj)
						return
					elif event.key == K_UP:
						paddle1.vel = -3
					elif event.key == K_DOWN:
						paddle1.vel = 3
				elif event.type == KEYUP:
					if event.key == K_UP:
						paddle1.vel = 0
					elif event.key == K_DOWN:
						paddle1.vel = 0

				
			if not one_player:
				if ball.vel[0] > 0:
					if paddle2.pos[1] < int(HEIGHT/2):
						paddle2.vel = DIFF
					elif paddle2.pos[1] > int(HEIGHT/2):
						paddle2.vel = -DIFF
				elif ball.vel[0] < 0:
					if paddle2.pos[1] < ball.pos[1]:
						paddle2.vel = DIFF
					else :
						paddle2.vel = -DIFF
			pygame.display.update()
			fpsClock.tick(60)

new_game(window_obj)


SystemExit: 